In [12]:
import pandas as pd
import numpy as np
df = pd.read_excel("./xlsx/join_data.xlsx")
df.columns
df = df[["DateTime", "종가_ex",'1Y_Mid_irs','10Y_Mid_irs','1Y_Mid_crs','국고1년','통안364일' ,'Mid_ndf','M1_스왑포인트']]
df["M1_스왑포인트"] = df["M1_스왑포인트"]/100
df.head()
df = df.set_index("DateTime")

In [13]:
df["1Y_Mid_irs대비"] =df["1Y_Mid_irs"]-df["1Y_Mid_irs"].shift(1)
df["10Y_Mid_irs대비"] =df["10Y_Mid_irs"]-df["10Y_Mid_irs"].shift(1)
df["1Y_Mid_crs대비"] =df["1Y_Mid_crs"]-df["1Y_Mid_crs"].shift(1)
df["국고1년대비"] =df["국고1년"]-df["국고1년"].shift(1)
df["통안364일대비"] = df["통안364일"] - df["통안364일"].shift(1)
df["Mid_ndf대비"] = df["Mid_ndf"] - df["Mid_ndf"].shift(1)
df["M1_스왑포인트대비"] = df["M1_스왑포인트"]- df["M1_스왑포인트"].shift(1)
df.head()

,종가_ex,1Y_Mid_irs,10Y_Mid_irs,1Y_Mid_crs,국고1년,통안364일,Mid_ndf,M1_스왑포인트,1Y_Mid_irs대비,10Y_Mid_irs대비,1Y_Mid_crs대비,국고1년대비,통안364일대비,Mid_ndf대비,M1_스왑포인트대비
DateTime,,,,,,,,,,,,,,,
2012-08-01,1126.5,2.85,2.91,2.09,2.82,2.80,1132.25,2.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-08-02,1131.7,2.82,2.86,2.08,2.79,2.77,1134.00,2.5,-0.03,-0.05,-0.01,-0.03,-0.03,1.75,-0.1
2012-08-03,1134.8,2.79,2.84,2.07,2.77,2.75,1138.00,2.4,-0.03,-0.02,-0.01,-0.02,-0.02,4.00,-0.1
2012-08-06,1129.0,2.81,2.85,2.07,2.78,2.76,1128.50,2.4,0.02,0.01,0.00,0.01,0.01,-9.50,0.0
2012-08-07,1128.8,2.82,2.87,2.09,2.77,2.75,1129.00,2.5,0.01,0.02,0.02,-0.01,-0.01,0.50,0.1


In [14]:
df["전날 종가_ex"] = df["종가_ex"].shift(1)
df = df.dropna()
df.columns


Index(['종가_ex', '1Y_Mid_irs', '10Y_Mid_irs', '1Y_Mid_crs', '국고1년', '통안364일',
       'Mid_ndf', 'M1_스왑포인트', '1Y_Mid_irs대비', '10Y_Mid_irs대비', '1Y_Mid_crs대비',
       '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트대비', '전날 종가_ex'],
      dtype='object')

In [15]:
# 종속변수 독립변수 설정
df_Target = df["종가_ex"]
df_Data = df[['1Y_Mid_irs대비', '10Y_Mid_irs대비','1Y_Mid_crs대비', '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트','전날 종가_ex']]
df_Data.feature= df_Data.columns
df_Data.feature


C:\Users\hyeok\AppData\Local\Temp\ipykernel_37436\3284639614.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_Data.feature= df_Data.columns


Index(['1Y_Mid_irs대비', '10Y_Mid_irs대비', '1Y_Mid_crs대비', '국고1년대비', '통안364일대비',
       'Mid_ndf대비', 'M1_스왑포인트', '전날 종가_ex'],
      dtype='object')

In [16]:
# 스케일링 코드
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(df_Data)
df_Data_scaled= scaler.transform(df_Data)
df_scaled = pd.DataFrame (data = df_Data_scaled, columns=df_Data.feature)
df_scaled.index = df_Target.index
df_Data_scaled

array([[0.48743719, 0.4744898 , 0.41975309, ..., 0.50091111, 1.        ,
        0.37153652],
       [0.48743719, 0.48979592, 0.41975309, ..., 0.50312731, 0.98571429,
        0.38790932],
       [0.51256281, 0.50510204, 0.42592593, ..., 0.48983009, 0.98571429,
        0.39767003],
       ...,
       [0.49748744, 0.46938776, 0.43209877, ..., 0.4986949 , 0.56428571,
        0.94175063],
       [0.51758794, 0.53571429, 0.45061728, ..., 0.49475499, 0.55      ,
        0.95969773],
       [0.46733668, 0.43877551, 0.35802469, ..., 0.48909136, 0.55714286,
        0.90554156]])

In [17]:
import statsmodels.api as sm
feature_add_std = sm.add_constant(df_scaled, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add_std)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 3.158e+04
Date:                Fri, 19 Aug 2022   Prob (F-statistic):               0.00
Time:                        13:25:21   Log-Likelihood:                -7660.3
No. Observations:                2459   AIC:                         1.534e+04
Df Residuals:                    2450   BIC:                         1.539e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          1011.9424      4.671    216.647      0.000    1002.783    1021.102
1Y_Mid_irs대비     12.7194      9.743      1.306      0.192      -6.386      31.824
10Y_Mid_irs대비    -5.6679      7.053     -0.804      0.422     -19.497       8.162
1Y_Mid_crs대비    -50.5227      3.664    -13.788      0.000     -57.708     -43.337
국고1년대비            5.7634      1.803      3.197      0.001       2.229       9.298
통안364일대비          0.3208      3.586      0.089      0.929      -6.711       7.353
Mid_ndf대비        26.6890      5.268      5.066      0.000      16.359      37.019
M1_스왑포인트         -0.8965      0.755     -1.188      0.235      -2.377       0.584
전날 종가_ex        316.3882      0.684    462.666      0.000     315.047     317.729
==============================================================================
Omnibus:                       44.133   Durbin-Watson:                   2.121
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               87.644
Skew:                           0.009   Prob(JB):                     9.29e-20
Kurtosis:                       3.925   Cond. No.                         180.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
# p값높은 컬럼들 제외
df_Target = df["종가_ex"]
df_scaled = df_scaled[['1Y_Mid_crs대비', '국고1년대비', 'Mid_ndf대비','전날 종가_ex']]
feature_add_std = sm.add_constant(df_scaled, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add_std)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 6.318e+04
Date:                Fri, 19 Aug 2022   Prob (F-statistic):               0.00
Time:                        13:25:22   Log-Likelihood:                -7661.9
No. Observations:                2459   AIC:                         1.533e+04
Df Residuals:                    2454   BIC:                         1.536e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         1014.7394      3.128    324.433      0.000    1008.606    1020.873
1Y_Mid_crs대비   -50.2383      3.651    -13.759      0.000     -57.398     -43.079
국고1년대비           6.0281      1.639      3.678      0.000       2.814       9.242
Mid_ndf대비       26.5643      5.266      5.045      0.000      16.239      36.890
전날 종가_ex       316.7256      0.632    501.199      0.000     315.486     317.965
==============================================================================
Omnibus:                       45.348   Durbin-Watson:                   2.123
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.101
Skew:                           0.009   Prob(JB):                     1.65e-20
Kurtosis:                       3.943   Cond. No.                         71.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [27]:
X_train, X_test, y_train, y_test = train_test_split(df_scaled,df_Target,test_size=0.2, random_state=16 )
lr = LinearRegression()
lr.fit (X_train, y_train)
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)
rmse,mse

(5.121878719967341, 26.233641622054282)

In [28]:
from sklearn.model_selection import cross_val_score
neg_mse_scores = cross_val_score(lr, df_scaled, df_Target,scoring="neg_mean_squared_error", cv = 5)
rmse_scores = np.sqrt(-1 *neg_mse_scores)
avg_rmse = np.mean(rmse_scores)
avg_rmse,rmse_scores, neg_mse_scores

(8.131060132846214,
 array([ 4.0964659 ,  6.74277563,  5.72018172, 18.98432984,  5.11154757]),
 array([ -16.78103284,  -45.46502325,  -32.72047892, -360.40477941,
         -26.1279186 ]))